In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

#regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statistics import *
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

#clustering
import sklearn as skl
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import metrics

!pip install statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf




In [ ]:
prec = pd.read_csv('precipitacionsbarcelonadesde1786.csv')
prec.head()

In [ ]:
#rename columns
prec.rename(columns={'Precip_Acum_Gener': "1",
                   'Precip_Acum_Febrer': "2",
                   'Precip_Acum_Marc': "3",
                   'Precip_Acum_Abril': "4",
                   'Precip_Acum_Maig': "5",
                   'Precip_Acum_Juny': "6",
                   'Precip_Acum_Juliol': "7",
                   'Precip_Acum_Agost': "8",
                   'Precip_Acum_Setembre': "9",
                   'Precip_Acum_Octubre': "10",
                   'Precip_Acum_Novembre': "11",
                   'Precip_Acum_Desembre': "12"},inplace=True)

In [ ]:
prec.columns

In [ ]:
prec.plot(x= 'Any',figsize=(15, 5))

In [ ]:
years = prec['Any'].unique()
df = pd.DataFrame(years, columns = ['Any'])
df['id'] = prec.index + 1
df.head(2)

In [ ]:
df = prec.merge(df, how='left',on='Any')

In [ ]:
df

In [ ]:
prec.plot(x= 'Any',figsize=(15, 5))

In [ ]:
# Creates a pivot table dataframe
table = df.melt(id_vars=['Any', 'id'], value_vars=['1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', '12'])


In [ ]:
table.rename(columns = {'value': "Total_precipitació"})

In [ ]:
# després de veure les cancel-lacions per mesos, cambio la data i utilitzo un datatime per unificar-la
table["Data"] = pd.to_datetime(dict(year=table["Any"], month=table["variable"], day="1"))


In [ ]:
table

In [ ]:
prom = table.groupby(['Any','variable']).mean()
prom

In [ ]:
sns.lineplot(data=prom, x='Any', y='value')


In [ ]:
temp = pd.read_csv('temperaturesbarcelonadesde1780.csv')
temp.head()

In [ ]:
#rename columns
temp.rename(columns={'Temp_Mitjana_Gener': "1",
                   'Temp_Mitjana_Febrer': "2",
                   'Temp_Mitjana_Marc': "3",
                   'Temp_Mitjana_Abril': "4",
                   'Temp_Mitjana_Maig': "5",
                   'Temp_Mitjana_Juny': "6",
                   'Temp_Mitjana_Juliol': "7",
                   'Temp_Mitjana_Agost': "8",
                   'Temp_Mitjana_Setembre': "9",
                   'Temp_Mitjana_Octubre': "10",
                   'Temp_Mitjana_Novembre': "11",
                   'Temp_Mitjana_Desembre': "12"},inplace=True)

In [ ]:
years = temp['Any'].unique()
df = pd.DataFrame(years, columns = ['Any'])
df['id'] = prec.index + 1
df.head(2)

In [ ]:
df1 = temp.merge(df, how='left',on='Any')
df

In [ ]:
# Creates a pivot table dataframe
table1 = df1.melt(id_vars=['Any','id'], value_vars=['1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', '12'])

In [ ]:
# després de veure les cancel-lacions per mesos, cambio la data i utilitzo un datatime per unificar-la
table1["Data"] = pd.to_datetime(dict(year=table1["Any"], month=table1["variable"], day="1"))

In [ ]:
table1.rename(columns = {'value': "Total_temperatures"})

In [ ]:
data = pd.merge(table1, table, on= ('id', 'variable', 'Any','Data'), how='outer')
data.head()

In [ ]:
df_def = data.rename(columns={'value_x': "Temperatures", 'value_y':"Precipitacions", 'variable': "Mes"})

In [ ]:
df_def.columns

In [ ]:
print(f'Número de filas con missing values: {df_def.isnull().any(axis=1).mean()}')

In [ ]:
df_def.dtypes

In [ ]:
df_def['Mes'] = df_def['Mes'].astype('int64')

In [ ]:
df_def.sort_values(by='Temperatures',ascending=False)[['Any','Mes','Temperatures', 'Data']].head(10)

In [ ]:
df_def.sort_values(by='Precipitacions',ascending=False)[['Any','Mes','Precipitacions','Data']].head(10)

In [ ]:
total = df_def.groupby(['Any']).mean()
total

In [ ]:
#Correlation matrix
fig,ax = plt.subplots(figsize=(6,4))
corr = df_def.corr()
sns.set_theme(style="white")
cmap = sns.diverging_palette(230, 20, as_cmap=True)

matrix = np.triu(corr)
sns.heatmap(corr, annot=True, mask=matrix, cmap=cmap)
plt.show()

In [ ]:
plt.figure(figsize =(15, 4))
sns.lineplot(x=df_def['Any'],y=df_def['Precipitacions'], color='#008ae6')
x = df['Any'].unique()
plt.xticks(np.arange(1786, max(x), 10))

plt.title("Evolution of precipitation",fontsize=15)
plt.xlabel("Years",fontsize=12)
plt.ylabel("Precipitation in mm",fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize =(15, 4))
sns.lineplot(x=df_def['Any'],y=df_def['Temperatures'], color='#008ae6')
x = df['Any'].unique()
plt.xticks(np.arange(1786, max(x), 10))

plt.title("Evolution of temperatures",fontsize=15)
plt.xlabel("Years",fontsize=12)
plt.ylabel("Precipitation in mm",fontsize=12)
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
#statistics
results = adfuller(total["Temperatures"])
print(f'ADF Statistic: {results[0]}')
print(f'P-value: {results[1]}')
for key, value in results[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

if results[1]<=0.05:
    print("conclusiones: ======")
    print("Rechazamos la hipotésis nula")
    print("Los datos nos indican que son estacionarios")
else:
    print("Conclusiones: ======")
    print("No se puede rechazar la hipotésis nula")
    print("Los datos nos indican que NO son estacionarios")

La estadística de prueba de Dickey-Fuller es -0.3480791502246142. El valor p asociado a esta estadística es 0.9184134654306413.

Los valores críticos para diferentes niveles de significancia son los siguientes:

Nivel 1%: -3.459105583381277
Nivel 5%: -2.8741898504150574
Nivel 10%: -2.5735117958412097
Interpretación de los resultados:

El valor de la estadística de prueba de -0.3480791502246142 es mayor que los valores críticos para todos los niveles de significancia (1%, 5% y 10%). Esto sugiere que los datos de la serie temporal no son estacionarios.
El valor p de 0.9184134654306413 es mayor que el nivel de significancia de 0.05 (5%). Esto respalda aún más la no estacionariedad de los datos.
En conclusión, según la prueba de Dickey-Fuller, no hay suficiente evidencia para rechazar la hipótesis nula de no estacionariedad para los datos de la serie temporal proporcionados.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
decomposition = seasonal_decompose(total['Temperatures'],model='additive', extrapolate_trend='freq',period = int(len(total)/2) )

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.figure(figsize =(15, 4))
plt.plot(trend,label = 'Trend')
plt.plot(seasonal,label = 'Seasonal', color="#F2389E")
plt.plot(residual,label = 'Residual', color="#2B9553")
plt.legend(loc="upper left")
plt.show()

In [ ]:
#statistics

results = adfuller(total["Precipitacions"])
print(f'ADF Statistic: {results[0]}')
print(f'P-value: {results[1]}')
for key, value in results[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

if results[1]<=0.05:
    print("conclusiones: ======")
    print("Rechazamos la hipotésis nula")
    print("Los datos nos indican que son estacionarios")
else:
    print("Conclusiones: ======")
    print("No se puede rechazar la hipotésis nula")
    print("Los datos nos indican que NO son estacionarios")


La estadística de prueba de Dickey-Fuller es -11.066505124160058. El valor p asociado a esta estadística es 4.667187628199031e-20.

Los valores críticos para diferentes niveles de significancia son los siguientes:

Nivel 1%: -3.4584868856997004
Nivel 5%: -2.873918902362675
Nivel 10%: -2.573367247623359
Interpretación de los resultados:

El valor de la estadística de prueba de -11.066505124160058 es mucho menor que los valores críticos para todos los niveles de significancia (1%, 5% y 10%). Esto sugiere que los datos de la serie temporal son estacionarios.
El valor p extremadamente pequeño de 4.667187628199031e-20 es mucho menor que el nivel de significancia de 0.05 (5%). Esto respalda aún más la estacionariedad de los datos.
En conclusión, según la prueba de Dickey-Fuller, hay suficiente evidencia para rechazar la hipótesis nula de no estacionariedad para los datos de la serie temporal proporcionados. Por lo tanto, los datos se consideran estacionarios.

PRECIPITACIONES   estacionaria table

TEMPERATURA No estacionaria table1

In [ ]:
!pip install statsforecast

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, ETS, Naive
from statsforecast.arima import arima_string

In [ ]:
#In google colab we should install pmdarima in order to use it.
!pip install pmdarima

In [ ]:
table1["unique_id"]= "1"
table1 = table1.rename(columns = {'Data': "ds", 'value':"y"})

table1.head()

In [ ]:
from pmdarima.arima import auto_arima

In [ ]:
# División de los datos en entrenamiento y prueba. Para los datos de prueba usaremos los 60 últimos meses para realizar la prueba y evaluación del desempeño de nuestro modelo.
y_train_df = table1[table1.ds<='2017-01-01']
y_test_df = table1[table1.ds>'2017-01-01']
y_train_df.shape, y_test_df.shape

In [ ]:
season_length = 60
horizon = len(y_test_df)

In [ ]:
# llamamos al model
models = [
    AutoARIMA(season_length=season_length),
    ETS(season_length=season_length),
    Naive()
]

In [ ]:
sf = StatsForecast(
    df=y_train_df,
    models=models,
    freq='MS', 
    n_jobs=-1
)

In [322]:
Y_hat_df = sf.forecast(horizon)
Y_hat_df.head()

KeyboardInterrupt: 